# SCAL3 demo

This demo uses the [Rust prototype](src/README.md) via its foreign function interface (FFI).

## Installation

Ensure you have compiled the prototype with `cargo build --release`.

In [41]:
%pip install cbor2 fastecdsa

from ctypes import cdll, memmove, string_at
from cbor2 import dumps, loads
from fastecdsa import keys, curve, ecdsa
from fastecdsa.encoding.sec1 import SEC1Encoder
from fastecdsa.encoding.util import int_to_bytes
import secrets
import hmac
import hashlib

Note: you may need to restart the kernel to use updated packages.


In [42]:
lib = cdll.LoadLibrary("target/release/libscal3.so")

We let Rust allocate a request/response buffer with sufficient bytes.

In [43]:
size = lib.scal3_buffer_size()
buf = lib.scal3_buffer_allocate()

## Helper functions

In [44]:
def write_request(request):
    data = dumps(request)
    memmove(buf, data, len(data))
def read_response():
    return loads(string_at(buf, size))
def ecdh(sk, pk):
    pk = SEC1Encoder().decode_public_key(pk, curve.P256)
    return SEC1Encoder().encode_public_key(sk * pk, compressed=False)[1:33]

## Setup

Create a single key pair and a PRF key for the provider. In production, these would be HSM-backed.

In [45]:

sk_provider, pk_provider = keys.gen_keypair(curve.P256)
pk_provider = SEC1Encoder.encode_public_key(pk_provider)
k_provider = secrets.token_bytes(32)

## Enrolment

The subscriber creates a possession key pair and a PRF key. In production, these would be backed by mobile secure hardware.

In [46]:

sk_device, pk_device = keys.gen_keypair(curve.P256)
pk_device = SEC1Encoder.encode_public_key(pk_device)
k_subscriber = secrets.token_bytes(32)

The subscriber creates a registration request. In production, this request would additionally be signed using an (attested) possession public key.

In [47]:
write_request({
    'mask': hmac.digest(k_subscriber, b"123456", hashlib.sha256),
    'randomness': secrets.token_bytes(32),
    'provider': pk_provider,
})
status = lib.scal3_subscriber_register(buf)
response = read_response()
subscriber = response['subscriber']
verifier = response['verifier']

Upon processing the registration request, the provider verifies if it is formed in a valid way.

In [48]:
credential = {
    'verifier': verifier,
    'device': pk_device,
}
provider_state = credential | {
    'provider': pk_provider,
    'verifierSecret': ecdh(sk_provider, subscriber),
}
write_request(provider_state)
status = lib.scal3_provider_accept(buf)
response = read_response()

## Authentication

The provider creates a challenge over some data. The combination is shared with a subscriber.

In [49]:
challenge_data = b"ts=1743930934&nonce=000001"
write_request({
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256)
})
status = lib.scal3_provider_challenge(buf)
response = read_response()
challenge = response['challenge']

The subscriber enters their PIN bound to some client data and uses the possession factor to create a signature.

In [50]:
client_data = b'{"operation":"log-in","session":"68c9eeeddfa5fb50"}'
client_data_hash = hashlib.sha256(client_data).digest()
write_request(credential | {
    'mask': hmac.digest(k_subscriber, b"123456", hashlib.sha256),
    'randomness': secrets.token_bytes(32),
    'provider': pk_provider,
    'subscriber': subscriber,
    'challenge': challenge,
    'hash': client_data_hash,
})
authentication = lib.scal3_subscriber_authenticate(buf)
response = read_response()
digest = response['digest']

r, s = ecdsa.sign(digest, sk_device, prehashed=True)
write_request({ 'proof': int_to_bytes(r, 32) + int_to_bytes(s, 32) })
status = lib.scal3_subscriber_pass(authentication, buf)
response = read_response()
sender = response['sender']
pass_attempt = response['pass']

Upon receiving back the challenge and the authentication attempt data, the provider tries to prove authentication.

In [51]:
write_request(provider_state | {
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256),
    'hash': client_data_hash,
    'passSecret': ecdh(sk_provider, sender),
    'pass': pass_attempt
})
status = lib.scal3_provider_prove(buf)
transcript = read_response()

## Auditing

Anyone can verify proven authentication.

In [52]:
write_request(credential | transcript | { 'hash': client_data_hash })
status = lib.scal3_verify(buf)
response = read_response()

assert status == 0
assert response['result'] == 'verified'

## Teardown

Ensure to free the buffer after using it.

In [53]:
lib.scal3_buffer_free(buf)

0